# zad2 Jakub Iliński

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List
import torch
import numpy as np
from torch.nn import functional as F

PAPUGA = 'flax-community/papuGaPT2'
DEVICE = "cuda"

tokenizer = AutoTokenizer.from_pretrained(PAPUGA)
model = AutoModelForCausalLM.from_pretrained(PAPUGA).to(DEVICE)
model.device